Enter Team Member Names here (double click to edit):

Spencer Kaiser

________

#In Class Assignment Three
In the following assignment you will be asked to fill in python code and derivations for a number of different problems. Please read all instructions carefully and turn in the rendered notebook (or HTML of the rendered notebook) to blackboard before the end of class.

**Distance Students**: please finish this assignment in 2 hours and 30 minutes. Turn in before next class per the instructions on blackboard.

________________________________________________________________________________________________________

##Downloading the Document Data
Please run the following code to read in the "20 newsgroups" dataset from sklearn's data loading module.

In [92]:
from sklearn.datasets import fetch_20newsgroups_vectorized
import numpy as np

# this takes about 30 seconds to compute, read the next section while this downloads
ds = fetch_20newsgroups_vectorized(subset='train')

# this holds the continuous feature data (which is tfidf)
print 'features shape:', ds.data.shape # there are ~11000 instances and ~130k features per instance
print 'target shape:', ds.target.shape 
print 'range of target:', np.min(ds.target),np.max(ds.target)
print 'Data type is', type(ds.data), float(ds.data.nnz)/(ds.data.shape[0]*ds.data.shape[1])*100, '% of the data is non-zero'

features shape: (11314, 130107)
target shape: (11314,)
range of target: 0 19
Data type is <class 'scipy.sparse.csr.csr_matrix'> 0.121435315436 % of the data is non-zero


##Understanding the Dataset
Look at the description for the 20 newsgroups dataset at http://qwone.com/~jason/20Newsgroups/. You have just downloaded the "vectorized" version of the dataset, which means all the words inside the articles have gone through a transformation that binned them into 130 thousand features related to the words in them. You can ignore the information on TFIDF-just recognize that it is a means of converting text to a vector of data. 

**Question Set 1**:
- How many instances are in the dataset? 
- What does each instance represent? 
- How many classes are in the dataset and what does each class represent?
- Would you expect a classifier trained on this data would generalize to documents written in the past week? Why or why not?
- Is the data represented as a sparse or dense matrix?

___

<strong>Instances</strong>: Around ~18,000 (7k of which comprise the testing subset, that's why they aren't included above) and each instance is a document from a specific newsgroup (20 in total, distributed evenly)

<strong>Classes</strong>: 130,107 (including the target class) and the majority of these classes represent a TFIDF value of specific keywords which present in the article. These values relate to the target classifier which is a general category of the document.

<strong>Would you expect a classifier trained on this data would generalize to documents written in the past week?</strong> Yes and no... I would expect it to classify it if the article were a 'general' news document discussing a something that occurs on a weekly basis. If the article discusses something very unique (e.g., the german plane crash), it would likely incorrectly classify it because the words used in the article may also be unique. Because the categories are quite broad and because the training dataset is quite large, I would assume the majority of articles will get classified correctly.

<strong>Is the data represented as a sparse or dense matrix?</strong> The data is represented as a sparse matrix.

___

##Measures of Distance
In the following block of code, we isolate three instances from the dataset. The instanca `a` is from the group computer graphics, `b` is from from the group recreation autos, and `c` is from group recreation motorcycle. **Exercise for part 2**: Calculate the Euclidean distance, cosine distance, and Jaccard similarity between each pair of instances using the imported function below. Remember that the Jaccard distance is only for binary valued vectors, so convert vectors to binary using a threshold. **Question for part 2**: Which distance seems more appropriate to use for this data? Why?

In [168]:
from scipy.spatial.distance import cosine
from scipy.spatial.distance import euclidean
from scipy.spatial.distance import jaccard
import numpy as np

# get first instance (comp)
idx = 550
a = ds.data[idx].todense()
a_class = ds.target_names[ds.target[idx]]
print 'Instance A is from class', a_class

# get second instance (autos)
idx = 4000
b = ds.data[idx].todense()
b_class = ds.target_names[ds.target[idx]]
print 'Instance B is from class', b_class

# get third instance (motorcycle)
idx = 7000
c = ds.data[idx].todense()
c_class = ds.target_names[ds.target[idx]]
print 'Instance C is from class', c_class

#Euclidean Distance
eucAB = euclidean(a,b);
eucAC = euclidean(a,c);
eucBC = euclidean(b,c);

#Cosine Distance
cosAB = cosine(a,b);
cosAC = cosine(a,c);
cosBC = cosine(b,c);

#Jaccard Distance
a_avg = np.average(a);
b_avg = np.average(b);
c_avg = np.average(c);

a_binary = a >= a_avg;
b_binary = b >= b_avg;
c_binary = c >= c_avg;

jacAB = jaccard(a_binary, b_binary);
jacAC = jaccard(a_binary, c_binary);
jacBC = jaccard(b_binary, c_binary);


# Enter distance comparison below for each pair of vectors:
p = 'Placeholder'
print '\n\nEuclidean Distance\n ab:', eucAB, 'ac:', eucAC, 'bc:',eucBC
print 'Cosine Distance\n ab:', cosAB, 'ac:', cosAC, 'bc:', cosBC
print 'Jaccard Dissimilarity (vectors should be boolean values)\n ab:', jacAB, 'ac:', jacAC, 'bc:', jacBC

print '\n\nBecause the target value of B and C are so related (Auto & Motorcycle), Jaccard distance may not be a good estimation of ditance. '
print '\nThat being said, either of the other two methods would be appropriate.'

Instance A is from class comp.graphics
Instance B is from class rec.autos
Instance C is from class rec.motorcycles


Euclidean Distance
 ab: 1.09851846719 ac: 1.18914054254 bc: 0.917779422666
Cosine Distance
 ab: 0.603371411376 ac: 0.707027614956 bc: 0.421159534335
Jaccard Dissimilarity (vectors should be boolean values)
 ab: 0.882113821138 ac: 0.875471698113 bc: 0.908794788274


Because the target value of B and C are so related (Auto & Motorcycle), Jaccard distance may not be a good estimation of ditance. 

That being said, either of the other two methods would be appropriate.


___
##Using scikit-learn with KNN
Now let's use stratified cross validation with a holdout set to train a KNN model in `scikit-learn`. Use the example below to train a KNN classifier. The documentation for KNeighbors is here: http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html  

**Exercise for part 3**: Use the code below to test what value of `n_neighbors` works best for the given data. *Note: do NOT change the metric to be anything other than `'euclidean'`. Other distance functions are not optimized for the amount of data we are working with.* **Question for part 3**: What is the accuracy of the best classifier you can create for this data (by changing only the `n_neighbors` parameter)? 

In [112]:
from sklearn.cross_validation import StratifiedShuffleSplit
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

cv = StratifiedShuffleSplit(ds.target, n_iter = 1, test_size = 0.5, train_size=0.5)

for trainidx, testidx in cv:
    # note that these are sparse matrices
    X_train = ds.data[trainidx] 
    X_test = ds.data[testidx] 
    y_train = ds.target[trainidx]
    y_test = ds.target[testidx]

k = 1;
highestAcc = 0.0;
kVal = 1;
while k <= 10:
    clf = KNeighborsClassifier(n_neighbors=5, weights='uniform', metric='euclidean')
    
    clf.fit(X_train, y_train)

    acc = clf.score(X_test,y_test);
    
    if acc > highestAcc:
        highestAcc = acc;
        kVal = k;
    
    k += 1;


print 'Accuracy of classifier is:', highestAcc * 100,"% with k of", kVal;

Accuracy of classifier is: 49.2663956161 % with k of 1


**Question for part 3**: With sparse data, does the use of a KDTree representation make sense? Why or Why not?

Yes. In general, sparser data leads to faster query times when KDTrees are used.

[Source: http://scikit-learn.org/stable/modules/neighbors.html#choice-of-nearest-neighbors-algorithm]
___

_____
##KNN extensions - Centroids
Now lets look at a very closely related classifier to KNN, called nearest centroid. In this classifier (which is more appropriate for big data scenarios and sparse data), the training step is used to calculate the centroids for each class. These centroids are saved. Unknown attributes, at prediction time, only need to have distances calculated for each saved centroid, drastically decreasing the time required for a prediction. **Exercise for part 4**: Use the template code below to create a nearest centroid classifier. Test which metric has the best performance: Euclidean, Cosine, or Manhattan. In `scikit-learn` you can see the documentation for NearestCentroid here: 
- http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.NearestCentroid.html#sklearn.neighbors.NearestCentroid

and for supported distance metrics here:
- http://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.distance_metrics.html#sklearn.metrics.pairwise.distance_metrics

In [120]:
from sklearn.neighbors.nearest_centroid import NearestCentroid

# the parameters for the nearest centroid metric to test are:
#    l1, l2, and cosine (all are optimized)

clf_e = NearestCentroid(metric='euclidean')
clf_e.fit(X_train, y_train);
e_acc = clf_e.score(X_test, y_test);

clf_c = NearestCentroid(metric='cosine')
clf_c.fit(X_train, y_train);
c_acc = clf_c.score(X_test, y_test);

clf_m = NearestCentroid(metric='manhattan')
clf_m.fit(X_train, y_train);
m_acc = clf_m.score(X_test, y_test);

acc = e_acc;
type = "Euclidean";

if c_acc > acc:
    acc = c_acc;
    type = "Cosine";
    
if m_acc > acc:
    acc = m_acc;
    type = "Manhattan";
    


print 'The best distance metric is:', type, "with an accuracy of", acc * 100, "%.";

The best distance metric is: Cosine with an accuracy of 49.1956867598 %.


___
##Naive Bayes Classification
Now let's look at the use of the Naive Bayes classifier. The 20 newsgroups dataset has 20 classes and about 130,000 features. Recall that the Naive Bayes classifer calculates a posterior distribution for each possible class. Each posterior distribution is a multiplication of many conditional distributions. **Question for part 5**: With this many classes and features, how many different conditional probabilities need to be parameterized? How many priors need to be modeled?

The number of conditional probabilities that need to be parameterized is equal to the number of classes (priors) multiplied by the number of features, which is around 260,000.

We will need to model 20 priors.

In [122]:
# Use this space for any calculations you might want to do
130107 * 20

2602140

___
##Naive Bayes in Scikit-learn
Scikit has several implementations of the Naive Bayes classifier: `GaussianNB`, `MultinomialNB`, and `BernoulliNB`. Look at the documentation here: http://scikit-learn.org/stable/modules/naive_bayes.html Take a look at each implementation and then answer this question: **Question for part 6**: If my instances contain continuous attributes, would it be better to use Gaussian Naive Bayes, Multinomial Naive Bayes, or Bernoulli? And Why? What if the data is sparse, does this change your answer? Why or Why not?

___
Because the attributes are not binary values (continuous attributes), Bernoulli is more than likely not the right choice as it requires binary valued vectors and converting to binary would result in a loss in precision. Either of the other two algorithms could be used (regardless of whether the data is sparse), however, because Multinomial Naive Bayes is frequently used in the context of text analysis (including TFIDF), using this algorithm would be a likely be the best choice.
___

##Naive Bayes Comparison
For the final section of this notebook let's compare the performance of Naive Bayes for document classification. Look at the parameters for `MultinomialNB`, and `BernoulliNB` (especially `alpha` and `binarize`). **Exercise for part 7**: Using the example code below, change the parameters for each classifier and see how accurate you can make the classifiers on the test set. **Question for part 7**: Why are these implementations so fast to train? What does the 'alpha' value control in these models (*i.e.*, how does it change the learned models)? 

In [167]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB


clf_mnb = MultinomialNB(alpha=0.001)
clf_bnb = BernoulliNB(alpha=0.00000001, binarize=0.0)

# fill in your code here
clf_mnb.fit(X_train, y_train);
clf_bnb.fit(X_train, y_train);

mnb_acc = clf_mnb.score(X_test, y_test);

bnb_acc = clf_bnb.score(X_test, y_test);

print "MNB Accuracy:", mnb_acc *100, "%."
print "BNB Accuracy:", bnb_acc *100, "%."


#Naive Bayes learners and classifiers can be extremely fast compared to more sophisticated methods. The decoupling of the class conditional feature distributions means that each distribution can be independently estimated as a one dimensional distribution. This in turn helps to alleviate problems stemming from the curse of dimensionality.

print '\nThese classifiers are so fast because they decouple class conditional feature distributions which allows for independent distribution estimation.';
print '\nThe alpha values control how the algorithm smoothens the data.';


MNB Accuracy: 88.5981969242 %.
BNB Accuracy: 84.58546933 %.

These classifiers are so fast because they decouple class conditional feature distributions which allows for independent distribution estimation.

The alpha values control how the algorithm smoothens the data.


________________________________________________________________________________________________________

That's all! Please **upload your rendered notebook to blackboard** and please include **team member names** in the notebook submission.